In [3]:
import polars as pl
import numpy as np

import mlflow as ml

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc

from xgboost import XGBClassifier, plot_importance

## Load data

In [4]:
df = pl.read_csv("data/customer_churn_telecom_services.csv")